In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
hyper_epoch = 100

In [2]:
for _ in range(hyper_epoch):
    # model parameters
    reuse = False
    block = MEGNet_block
    head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update])
    data = np.random.choice(['../Data/{}_data_ACSF_expand.pickle',\
                             '../Data/{}_data_SOAP_expand.pickle'])
    batch_size = 32
    dim = int(np.random.choice([128,256]))
    epochs = 50
    clip = 0.4
    layer1 = int(np.random.choice([3,4,5]))
    layer2 = int(np.random.choice([3,4,5]))
    factor = int(np.random.choice([2,3]))
    lr = 1e-4

    train_dl,val_dl = get_data(data,batch_size)

    model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

    model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,scheduler=scheduler)

    save_results(train_loss_list,val_loss_list,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)
    save_model_type(bestWeight,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)